<a href="https://colab.research.google.com/github/lewisnjue/AI/blob/main/knowledge_distilation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets




In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'

print(f"using {device} device")

using cuda device


the input images are grb , so they have 3 channels and are 32 x 32 ,e ach images is descrbed 3072 numbers ranging from 0 to 255. a common practice in neural networks is to normalize he input. which is done for mulipole reaasons , including avoiding sturaion in commonly used activation function and increasing numerical stability. our normalization process consists of subracting the means and dividing by he standrd deviation alogn each channel.

In [6]:
# Below we are preprocessing data for CIFAR-10. We use an arbitrary batch size of 128.
transforms_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Loading the CIFAR-10 dataset:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_cifar)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_cifar)

100%|██████████| 170M/170M [00:14<00:00, 12.0MB/s]


In [10]:
#Dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

In [7]:
# Deeper neural network class to be used as teacher:
class DeepNN(nn.Module):
    def __init__(self, num_classes=10):
        super(DeepNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Lightweight neural network class to be used as student:
class LightNN(nn.Module):
    def __init__(self, num_classes=10):
        super(LightNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [8]:
def train(model, train_loader, epochs, learning_rate, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # inputs: A collection of batch_size images
            # labels: A vector of dimensionality batch_size with integers denoting class of each image
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            # outputs: Output of the network for the collection of images. A tensor of dimensionality batch_size x num_classes
            # labels: The actual labels of the images. Vector of dimensionality batch_size
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

def test(model, test_loader, device):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

In [11]:
torch.manual_seed(42)
nn_deep = DeepNN(num_classes=10).to(device)
train(nn_deep, train_loader, epochs=10, learning_rate=0.001, device=device)
test_accuracy_deep = test(nn_deep, test_loader, device)

# Instantiate the lightweight network:
torch.manual_seed(42)
nn_light = LightNN(num_classes=10).to(device)

Epoch 1/10, Loss: 1.3282010235140085
Epoch 2/10, Loss: 0.85995360743969
Epoch 3/10, Loss: 0.6740342608040861
Epoch 4/10, Loss: 0.5377977293013306
Epoch 5/10, Loss: 0.41822918742666465
Epoch 6/10, Loss: 0.31140046282802397
Epoch 7/10, Loss: 0.22527728426029614
Epoch 8/10, Loss: 0.17463002896979643
Epoch 9/10, Loss: 0.13691106591078325
Epoch 10/10, Loss: 0.12057654050362232
Test Accuracy: 75.48%


In [12]:
torch.manual_seed(42)
new_nn_light = LightNN(num_classes=10).to(device)

In [13]:
# Print the norm of the first layer of the initial lightweight model
print("Norm of 1st layer of nn_light:", torch.norm(nn_light.features[0].weight).item())
# Print the norm of the first layer of the new lightweight model
print("Norm of 1st layer of new_nn_light:", torch.norm(new_nn_light.features[0].weight).item())

Norm of 1st layer of nn_light: 2.327361822128296
Norm of 1st layer of new_nn_light: 2.327361822128296


In [14]:
total_params_deep = "{:,}".format(sum(p.numel() for p in nn_deep.parameters()))
print(f"DeepNN parameters: {total_params_deep}")
total_params_light = "{:,}".format(sum(p.numel() for p in nn_light.parameters()))
print(f"LightNN parameters: {total_params_light}")

DeepNN parameters: 1,186,986
LightNN parameters: 267,738


In [15]:
train(nn_light, train_loader, epochs=10, learning_rate=0.001, device=device)
test_accuracy_light_ce = test(nn_light, test_loader, device)

Epoch 1/10, Loss: 1.4709399301377708
Epoch 2/10, Loss: 1.1618167847928489
Epoch 3/10, Loss: 1.030767710312553
Epoch 4/10, Loss: 0.926373452939036
Epoch 5/10, Loss: 0.8455027415014594
Epoch 6/10, Loss: 0.7781399855833224
Epoch 7/10, Loss: 0.7102214229076415
Epoch 8/10, Loss: 0.6532236542695623
Epoch 9/10, Loss: 0.6001846083747152
Epoch 10/10, Loss: 0.5505389866164273
Test Accuracy: 71.08%


**teh big picture**

you have:
- a teacher model: big , accurate and already trained
- a student model : smaller , less accurate (yet) , but faster adn lighter.

the goal is to train the student so that it learns not only from the ground truth lables( like usual cross entropy loss) but also from the teacher ouput, wich containins  richer information.

**why bother learning fomr the teacher ouputs?**
because:
- a well trined teacher doest just ouput it a truct with 99 centrary
- it also tellyou , `i kinda see it also similar to an automobile , and maybe a litle bit like an airplaine`

these subtle probability distributions reflex relationships between classes. the student can learn from these relationships, not just from hard ground truth lables.


**the problem with cross-entropy alone**

regular cross-entropy loss lokks at one label. it hard(eg this is truact, 100% centrainty)
it ignores the naunced probabilites the teacher can offer.
but ifyou look at the teacher entire softmax output, you can get soft lables like
```makefile
truck: 0.7
automobile: 0.2
airplane: 0.1
dog: 0.0
```
this says more about how teeacher understands the world


**how knowledge distillaiotn works**

you combine two losses:
1. cross-entropy loss(CE) with the real labels ( like usual classification)
2. distillation loss: comparing the studnet ouptu to the teacher soft ouputs.


the distillation loss helps the student mimic the teacher probability distribution , not jsut match the hard labels.


**what do these terms mean?**

1. *Temperature(T)*
-this is a parameter used to smooth the sofmax ouput from both teaacher and student.
- higher T -> smoother distributions-> small probability get bigger so the student pay attention to more than just the top prediction.
example with T=1

```less

Teacehr softmax:[0.95,0.04,0.01]

```
with T=5

```less
Teacher softmax: [0.5,0.04,0.01]

```

smoother -> easier for the student to learn relative similarites between classes.


2. soft_target_less_weight

- this is the weight you assing to the distribution loss .
- it controls how much the student focuses on mimicking the teacher soft ouputs.

3. ce_loss_weight

- this is teh weight for the normal cross-entropy loss with the true lables.
- it controls how much the student focuses on learning the real lables.


**what the pint of balancing these?**

you tuen soft_target_loss_weight and ce_loss_wight dependin on:
- how much you trust the teacher.
- how much you still want to respect the real lables.

you ca think of it like:


`Total Loss = ce_loss_weight * CrossEntropyLoss + soft_target_loss_weight * DistillationLoss`



In [16]:
def train_knowledge_distillation(teacher, student, train_loader, epochs, learning_rate, T, soft_target_loss_weight, ce_loss_weight, device):
    ce_loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(student.parameters(), lr=learning_rate)

    teacher.eval()  # Teacher set to evaluation mode
    student.train() # Student to train mode

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass with the teacher model - do not save gradients here as we do not change the teacher's weights
            with torch.no_grad():
                teacher_logits = teacher(inputs)

            # Forward pass with the student model
            student_logits = student(inputs)

            #Soften the student logits by applying softmax first and log() second
            soft_targets = nn.functional.softmax(teacher_logits / T, dim=-1)
            soft_prob = nn.functional.log_softmax(student_logits / T, dim=-1)

            # Calculate the soft targets loss. Scaled by T**2 as suggested by the authors of the paper "Distilling the knowledge in a neural network"
            soft_targets_loss = torch.sum(soft_targets * (soft_targets.log() - soft_prob)) / soft_prob.size()[0] * (T**2)

            # Calculate the true label loss
            label_loss = ce_loss(student_logits, labels)

            # Weighted sum of the two losses
            loss = soft_target_loss_weight * soft_targets_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

# Apply ``train_knowledge_distillation`` with a temperature of 2. Arbitrarily set the weights to 0.75 for CE and 0.25 for distillation loss.
train_knowledge_distillation(teacher=nn_deep, student=new_nn_light, train_loader=train_loader, epochs=10, learning_rate=0.001, T=2, soft_target_loss_weight=0.25, ce_loss_weight=0.75, device=device)
test_accuracy_light_ce_and_kd = test(new_nn_light, test_loader, device)

# Compare the student test accuracy with and without the teacher, after distillation
print(f"Teacher accuracy: {test_accuracy_deep:.2f}%")
print(f"Student accuracy without teacher: {test_accuracy_light_ce:.2f}%")
print(f"Student accuracy with CE + KD: {test_accuracy_light_ce_and_kd:.2f}%")

Epoch 1/10, Loss: 2.3757467836980015
Epoch 2/10, Loss: 1.8585280073268333
Epoch 3/10, Loss: 1.6373781712768634
Epoch 4/10, Loss: 1.4800155147567124
Epoch 5/10, Loss: 1.3542415176511116
Epoch 6/10, Loss: 1.2409376774907417
Epoch 7/10, Loss: 1.1423496724394582
Epoch 8/10, Loss: 1.057148908715114
Epoch 9/10, Loss: 0.9775785663548637
Epoch 10/10, Loss: 0.9119357076447333
Test Accuracy: 70.94%
Teacher accuracy: 75.48%
Student accuracy without teacher: 71.08%
Student accuracy with CE + KD: 70.94%
